<a href="https://colab.research.google.com/github/marriamaslova/compling_nlp_hse_course/blob/master/notebooks/rlhf/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1 (10 баллов)

Вам нужно воспроизвести пайплайн дообучения из семинара, используя русскоязычные модели. То есть, вам нужно дообучить генеративную модель генерировать более положительные тексты с помощью модели определения тональности.

1. Вам потребуется модель определения тональности для русского языка. Вы можете найти ее на huggingface или обучить самостоятельно. В прошлых семинарах мы пользовались датасетом с токсичными текстами - можно использовать его для обучения (или любой другой датасет на русском языке). Удобнее всего будет обучать модель через huggingface. Можете взять за основу вот этот туториал - https://huggingface.co/docs/transformers/training#train-a-tensorflow-model-with-keras После обучения можете сохранить модель через model.save_pretrained(path) и потом использовать ее как любую другую модель из huggingface hub. 
Используйте не очень большую модель, чтобы все поместилось в колабе (например, distilbert-multilingual)

2. Генеративную модель в целом можно взять любую, но лучше всего будет модель, специально обученная на русском языке (rugpt). 

Изначальная генерация текстов

In [ ]:
! pip install transformers 
! pip install wandb

In [ ]:
! pip install datasets
! pip install trl

In [3]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset
import numpy as np
from tensorflow.keras.optimizers import Adam

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [4]:
config = PPOConfig(
    model_name="ai-forever/rugpt3small_based_on_gpt2",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [ ]:
dataset = load_dataset("csv", data_files="/content/labeled.csv")

In [8]:
def build_dataset(config, dataset=dataset['train'], input_min_text_length=2, input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = dataset
    ds = ds.rename_columns({"comment": "review"})
    
    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 2000, batched=False)
    
    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [ ]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [ ]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model (обратите внимание что это одна и так же модель изначально)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

In [11]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [12]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

Подготовка модели для sentiment analysis

Я очень долго пыталась обучить модель на собственном датасете, но как будто какой-то пример все время подавался неправильно и обучение не получалось (хотя я и размер примеров ограничивала, и многие параметры пробовала менять). В конце концов я сдалась и взяла предобученную модель.

In [ ]:
sentiment_pipe = pipeline("sentiment-analysis", model="cointegrated/rubert-tiny-sentiment-balanced", device=device)

In [14]:
text = "фильм отстой!"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'negative', 'score': 1.3824427127838135},
  {'label': 'neutral', 'score': -0.7409297823905945},
  {'label': 'positive', 'score': -0.6284522414207458}]]

In [15]:
text = "лучший фильм на этой планете!!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -3.0470917224884033},
  {'label': 'neutral', 'score': -0.6917519569396973},
  {'label': 'positive', 'score': 3.3792996406555176}]]

In [16]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, 
              "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

Обучение генеративной модели

In [ ]:
output_min_length = 8
output_max_length = 32
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), 
                         total=dataset.num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[2]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

In [ ]:
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', 5000)

In [ ]:
#### get a batch from the dataset
bs = 32
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[2]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[2]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

In [21]:
pd.DataFrame.from_dict(game_data)

,query,response (before),response (after),rewards (before),rewards (after)
0,Пффф. Тут у строителей,"плодотворная беседа произошла.\n\n— Ну, вы ту...",хороший торжественный банкет). Обалденный пам...,-1.138301,4.061328
1,В этих случаях ищут максимальные раз,ламы касательно образа жизни и поведения детей...,овые команды). Достаточно просто поздороватьс...,-0.651272,2.886158
2,Умеет,"человек, умеющий считать? Играет Соламидо",. И заставить меня приносить Ему Пачу: дово,-2.733760,0.234376
3,"Во, у",вас восточное направление. Давайте два участка,меня тоже потрясающая! Вот не знала,-1.903221,1.113222
4,вот ты мне только что написал тебя,"честно, ты -- самый лучший собеседник и я даж...",")) ) ведь я тебе сегодня помогала, только у ме...",1.724876,2.335308
5,СЕГОДНЯ ОНИ ОТ,РЕШИЛИСЬ ОТ ОБЕСПЕЧЕНИЯ ФИГАЛИМЕНТА.\n\nПОЧЕМУ...,НОШЛИ ВСЯ ЕСЛИ ГОСПОДА СОТВОРИЛИ ПОБЕДИТЬ ВОЙН...,0.389522,-1.161602
6,Ну а что такого,", в конце концов? Слабодержцам,",", нормально, я девушка хорошая, а в итоге все",-2.452418,1.474446
7,Вы правы. Но,я никоим образом не изучала эти тайны. Способ...,"все же иногда стоит уступить, подарить друг д...",-1.378752,1.058647
8,"Ну,",веселая может быть я? – хмыкнула реплика Ильи...,серьёзно..))\n\nКак будем отмечать день рожде...,-3.455990,3.167920
9,Скримеры рак,елей (первоначально Г.Я. Зайончковский),"етами? :) кого взял, близких прикольных друзей...",-3.275602,1.904207


По скорам видно, что дообученная модель генерирует тексты, более близкие к положительной тональности.